# City Geographic Data ETL Project

This notebook demonstrates the ETL (Extract, Transform, Load) process for city geographic and quality-of-life data. We'll cover:
- Setting up database connections
- Loading and cleaning raw data
- Transforming and validating data
- Loading data into PostgreSQL

## 1. Setup and Dependencies

First, let's import required libraries and configure our database connection.

In [ ]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import logging
from datetime import datetime
from src.models.city_data_collector import CityDataCollector

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Initialize data collector
city_collector = CityDataCollector()

# Database connection with error handling
try:
    DATABASE_URL = os.getenv('DATABASE_URL')
    if not DATABASE_URL:
        raise ValueError("DATABASE_URL environment variable is not set")
        
    engine = create_engine(DATABASE_URL)
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    logger.info("Successfully connected to the database")
except Exception as e:
    logger.error(f"Database connection error: {str(e)}")
    raise

## 2. Create Sample Data

Let's create some sample city data to work with.

## 2. Collect Real City Data

Now we'll collect real data from various APIs:
- Basic city information from World Cities Database
- Quality of life metrics from Numbeo
- Environmental data from OpenWeatherMap

In [ ]:
# Initialize ETL process
from src.models.city_etl import CityDataETL

etl = CityDataETL(DATABASE_URL)

# List of cities to process
cities = [
    {'name': 'New York', 'country': 'US'},
    {'name': 'London', 'country': 'GB'},
    {'name': 'Tokyo', 'country': 'JP'},
    {'name': 'Paris', 'country': 'FR'},
    {'name': 'Singapore', 'country': 'SG'},
    {'name': 'Sydney', 'country': 'AU'},
    {'name': 'Dubai', 'country': 'AE'},
    {'name': 'Moscow', 'country': 'RU'},
    {'name': 'Berlin', 'country': 'DE'},
    {'name': 'Toronto', 'country': 'CA'},
    {'name': 'Barcelona', 'country': 'ES'},
    {'name': 'Amsterdam', 'country': 'NL'},
    {'name': 'Seoul', 'country': 'KR'},
    {'name': 'Mumbai', 'country': 'IN'},
    {'name': 'Sao Paulo', 'country': 'BR'}
]

# Run ETL process
df = etl.run_etl(cities)

# Display results
print("\nETL Process Results:")
print("-" * 50)
print(f"Total cities processed: {len(cities)}")
print(f"Successfully collected data for {len(df)} cities")
print("\nColumns with data:")
for column in df.columns:
    non_null = df[column].count()
    print(f"{column}: {non_null} values")

## 3. Analyze and Validate Collected Data

Let's analyze the data we collected to ensure quality and completeness:

In [ ]:
# Basic data quality checks
def analyze_data_quality(df):
    print("Data Quality Report:")
    print("-" * 50)
    
    # Check for missing values
    missing = df.isnull().sum()
    print("\nMissing values by column:")
    print(missing[missing > 0])
    
    # Check numeric columns statistics
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    print("\nNumeric columns statistics:")
    print(df[numeric_cols].describe())
    
    # Check for potential outliers
    print("\nPotential outliers (values beyond 3 standard deviations):")
    for col in numeric_cols:
        mean = df[col].mean()
        std = df[col].std()
        outliers = df[(df[col] > mean + 3*std) | (df[col] < mean - 3*std)][col]
        if not outliers.empty:
            print(f"\n{col}:")
            print(outliers)

# Analyze the collected data
analyze_data_quality(df_real)

# Visualize some key metrics
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.boxplot(data=df_real[[
    'happiness_index', 
    'health_index', 
    'cost_of_living_index', 
    'education_level_score'
]])
plt.title('Distribution of Key Quality of Life Metrics')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Natural Features Analysis

Let's analyze the natural features data we collected:

In [ ]:
# Analyze natural features data
natural_features_cols = [
    'distance_to_water',
    'distance_to_mountains',
    'distance_to_forest',
    'distance_to_park',
    'forest_proximity_score',
    'green_space_ratio'
]

print("Natural Features Statistics:")
print("-" * 50)
print("\nAverage distances (km):")
for col in natural_features_cols:
    if 'distance' in col:
        mean_val = df_real[col].mean()
        if not pd.isna(mean_val):
            print(f"{col}: {mean_val:.2f}")

print("\nGreen Space Metrics:")
print(f"Average forest proximity score: {df_real['forest_proximity_score'].mean():.2f}")
print(f"Average green space ratio: {df_real['green_space_ratio'].mean():.2f}")

# Visualize natural features data
plt.figure(figsize=(12, 6))
df_real[natural_features_cols].boxplot()
plt.title('Distribution of Natural Features Metrics')
plt.xticks(rotation=45)
plt.ylabel('Distance (km) / Score')
plt.tight_layout()
plt.show()

# Create a scatter plot comparing green space ratio to quality of life
plt.figure(figsize=(10, 6))
plt.scatter(df_real['green_space_ratio'], df_real['happiness_index'])
plt.xlabel('Green Space Ratio')
plt.ylabel('Happiness Index')
plt.title('Correlation between Green Space and Happiness')
for i, txt in enumerate(df_real['city_name']):
    plt.annotate(txt, (df_real['green_space_ratio'].iloc[i], df_real['happiness_index'].iloc[i]))
plt.tight_layout()
plt.show()

In [ ]:
# Create sample city data
raw_data = {
    'city_name': ['New York', 'London', 'Tokyo', 'Paris', 'Singapore',
                  'new york', 'LONDON', 'токио', 'paris ', 'Singapore'],
    'country': ['USA', 'UK', 'Japan', 'France', 'Singapore',
               'USA', 'UK', 'Japan', 'France', 'Singapore'],
    'population': [8400000, 8900000, 37400000, 2100000, 5700000,
                  np.nan, 8900000, 37400000, np.nan, 5700000],
    'quality_of_life': [75.5, 82.3, 80.1, 85.2, 90.0,
                       75.5, 82.3, 80.1, 85.2, np.nan],
    'coordinates': ['40.7128,-74.0060', '51.5074,-0.1278', '35.6762,139.6503',
                   '48.8566,2.3522', '1.3521,103.8198',
                   '40.7128,-74.0060', '', '', '48.8566,2.3522', ''],
}

df_raw = pd.DataFrame(raw_data)
print("Raw data sample:")
df_raw.head(10)

## 3. Data Cleaning and Transformation

Let's clean and transform our data:

In [ ]:
def clean_city_data(df):
    """Clean and transform city data"""
    df_clean = df.copy()
    
    # Standardize city names
    df_clean['city_name'] = df_clean['city_name'].str.strip().str.title()
    
    # Clean country names
    df_clean['country'] = df_clean['country'].str.strip()
    
    # Handle missing population data
    df_clean['population'] = df_clean.groupby('city_name')['population'].transform(
        lambda x: x.fillna(x.mean()))
    
    # Extract coordinates
    df_clean[['latitude', 'longitude']] = df_clean['coordinates'].str.split(',', expand=True)
    df_clean['latitude'] = pd.to_numeric(df_clean['latitude'], errors='coerce')
    df_clean['longitude'] = pd.to_numeric(df_clean['longitude'], errors='coerce')
    
    # Drop duplicates
    df_clean = df_clean.drop_duplicates(subset=['city_name', 'country'])
    
    return df_clean

# Clean the data
df_cleaned = clean_city_data(df_raw)
print("Cleaned data:")
df_cleaned.head()

## 4. Data Validation

Let's validate our cleaned data to ensure it meets our quality standards.

In [ ]:
def validate_city_data(df):
    """Validate cleaned city data and generate report"""
    validation_results = {
        'total_records': len(df),
        'unique_cities': df['city_name'].nunique(),
        'unique_countries': df['country'].nunique(),
        'missing_coordinates': df[df['latitude'].isna() | df['longitude'].isna()].shape[0],
        'missing_population': df['population'].isna().sum(),
        'quality_score_range': f"{df['quality_of_life'].min():.1f} - {df['quality_of_life'].max():.1f}"
    }
    
    # Print validation report
    print("Data Validation Report:")
    for key, value in validation_results.items():
        print(f"{key.replace('_', ' ').title()}: {value}")
        
    return validation_results

# Validate the cleaned data
validation_results = validate_city_data(df_cleaned)

## 5. Database Loading

Now let's load the cleaned and validated data into our PostgreSQL database.

In [ ]:
def load_to_database(df, engine):
    """Load data to PostgreSQL with transaction management"""
    try:
        with engine.begin() as connection:
            # Create temporary table for upserting
            df.to_sql('cities_temp', connection, if_exists='replace', index=False)
            
            # Perform upsert using a transaction
            upsert_query = """
            INSERT INTO cities (city_name, country, latitude, longitude, population)
            SELECT city_name, country, latitude, longitude, population
            FROM cities_temp
            ON CONFLICT (city_name, country)
            DO UPDATE SET
                latitude = EXCLUDED.latitude,
                longitude = EXCLUDED.longitude,
                population = EXCLUDED.population;
            """
            
            connection.execute(text(upsert_query))
            logger.info("Successfully loaded data to database")
            
    except Exception as e:
        logger.error(f"Error loading data to database: {str(e)}")
        raise

# Load data to database
load_to_database(df_cleaned, engine)

## 6. Verify Data Loading

Let's verify that our data was loaded correctly.

In [ ]:
# Query to verify loaded data
verification_query = """
SELECT 
    city_name,
    country,
    latitude,
    longitude,
    population
FROM cities
ORDER BY population DESC;
"""

# Execute query and display results
df_verified = pd.read_sql(verification_query, engine)
print("Verified data in database:")
df_verified

# City Data ETL Project

This notebook demonstrates the process of collecting, transforming, and analyzing city data using Python and PostgreSQL. We'll explore various aspects of urban life including:
- Cultural and social indicators
- Economic metrics
- Environmental factors
- Quality of life measurements

## 1. Setup and Dependencies

First, let's import all required libraries and set up our database connection.

In [ ]:
import os
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns

# Load environment variables
load_dotenv()

# Database connection
DATABASE_URL = os.getenv('DATABASE_URL')
engine = create_engine(DATABASE_URL)

## 2. Create Database Schema

Let's create our database schema using SQLAlchemy.

In [ ]:
# Create tables SQL
create_tables_sql = """
CREATE TABLE IF NOT EXISTS cities (
    id SERIAL PRIMARY KEY,
    city_name VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    latitude FLOAT CHECK (latitude BETWEEN -90 AND 90),
    longitude FLOAT CHECK (longitude BETWEEN -180 AND 180),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UNIQUE (city_name, country)
);

CREATE TABLE IF NOT EXISTS city_metrics (
    id SERIAL PRIMARY KEY,
    city_id INTEGER REFERENCES cities(id),
    timestamp TIMESTAMP NOT NULL,
    english_proficiency_score FLOAT CHECK (english_proficiency_score BETWEEN 0 AND 100),
    lgbt_acceptance_score FLOAT CHECK (lgbt_acceptance_score BETWEEN 0 AND 100),
    cultural_events_per_capita FLOAT CHECK (cultural_events_per_capita >= 0),
    education_level_score FLOAT CHECK (education_level_score BETWEEN 0 AND 100),
    happiness_index FLOAT CHECK (happiness_index BETWEEN 0 AND 100),
    sports_facilities_per_capita FLOAT CHECK (sports_facilities_per_capita >= 0),
    health_index FLOAT CHECK (health_index BETWEEN 0 AND 100),
    noise_pollution_level FLOAT CHECK (noise_pollution_level BETWEEN 0 AND 100),
    air_quality_index FLOAT CHECK (air_quality_index BETWEEN 0 AND 500),
    water_quality_score FLOAT,
    economic_growth_rate FLOAT,
    cost_of_living_index FLOAT,
    average_salary FLOAT,
    housing_price_index FLOAT,
    rent_price_index FLOAT,
    wind_speed_avg FLOAT,
    sunny_days_per_year INTEGER,
    distance_to_water FLOAT,
    distance_to_mountains FLOAT,
    forest_proximity_score FLOAT,
    green_space_ratio FLOAT,
    traffic_congestion_score FLOAT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

# Execute SQL to create tables
with engine.connect() as conn:
    conn.execute(text(create_tables_sql))
    conn.commit()

## 3. Data Collection

We'll collect data from various sources. For this example, we'll use:
- Numbeo API for cost of living and quality of life data
- OpenWeatherMap for weather data
- World Bank API for economic indicators

Note: You'll need to sign up for API keys for these services.

In [ ]:
def fetch_numbeo_data(city, country):
    """Simulate fetching data from Numbeo API"""
    # This is a mock function - you'll need to implement real API calls
    return {
        'cost_of_living_index': np.random.uniform(50, 150),
        'rent_index': np.random.uniform(20, 100),
        'health_care_index': np.random.uniform(40, 90),
        'quality_of_life_index': np.random.uniform(60, 180)
    }

def fetch_weather_data(city, country):
    """Simulate fetching weather data"""
    return {
        'wind_speed_avg': np.random.uniform(2, 15),
        'sunny_days': int(np.random.uniform(100, 300))
    }

# Sample cities
sample_cities = [
    {'name': 'New York', 'country': 'USA'},
    {'name': 'London', 'country': 'UK'},
    {'name': 'Tokyo', 'country': 'Japan'},
    {'name': 'Berlin', 'country': 'Germany'},
    {'name': 'Singapore', 'country': 'Singapore'}
]

# Collect data for each city
city_data = []
for city in sample_cities:
    numbeo_data = fetch_numbeo_data(city['name'], city['country'])
    weather_data = fetch_weather_data(city['name'], city['country'])
    
    city_data.append({
        'city_name': city['name'],
        'country': city['country'],
        **numbeo_data,
        **weather_data
    })

# Convert to DataFrame
df_raw = pd.DataFrame(city_data)
df_raw.head()

## 4. Data Transformation

Now let's clean and transform our collected data.

In [ ]:
def transform_city_data(df):
    """Transform and clean the raw city data"""
    # Create a copy to avoid modifying the original
    df_clean = df.copy()
    
    # Add timestamp
    df_clean['timestamp'] = datetime.now()
    
    # Normalize numerical columns to 0-100 scale
    numerical_cols = ['cost_of_living_index', 'rent_index', 'health_care_index', 'quality_of_life_index']
    for col in numerical_cols:
        if col in df_clean.columns:
            df_clean[col] = 100 * (df_clean[col] - df_clean[col].min()) / (df_clean[col].max() - df_clean[col].min())
    
    # Round numerical values
    df_clean = df_clean.round(2)
    
    return df_clean

# Transform the data
df_transformed = transform_city_data(df_raw)
df_transformed.head()

## 5. Load Data into Database

Now we'll load our transformed data into PostgreSQL.

In [ ]:
def load_cities(df, engine):
    """Load city data into the database with proper duplicate handling"""
    try:
        with engine.begin() as connection:
            # Create temporary table for upserting cities
            cities_df = df[['city_name', 'country', 'latitude', 'longitude']].drop_duplicates()
            cities_df.to_sql('cities_temp', connection, if_exists='replace', index=False)
            
            # Upsert cities using ON CONFLICT
            upsert_query = """
                INSERT INTO cities (city_name, country, latitude, longitude)
                SELECT city_name, country, latitude, longitude
                FROM cities_temp
                ON CONFLICT (city_name, country) 
                DO UPDATE SET
                    latitude = EXCLUDED.latitude,
                    longitude = EXCLUDED.longitude;
            """
            connection.execute(text(upsert_query))
            
    # Get city IDs
    city_ids = pd.read_sql(
        'SELECT id, city_name FROM cities',
        engine
    ).set_index('city_name')['id'].to_dict()
    
    # Prepare metrics data
    df['city_id'] = df['city_name'].map(city_ids)
    metrics_cols = ['city_id', 'timestamp', 'cost_of_living_index', 'rent_index',
                    'health_care_index', 'quality_of_life_index', 'wind_speed_avg',
                    'sunny_days']
    
    # Insert metrics
    df[metrics_cols].to_sql('city_metrics', engine, if_exists='append', index=False)

# Load the transformed data
load_cities(df_transformed, engine)

## 6. Analysis with SQL

Let's analyze our data using SQL queries with window functions and joins.

In [ ]:
# Complex SQL query using window functions and joins
analysis_query = """
WITH RankedCities AS (
    SELECT 
        c.city_name,
        c.country,
        cm.cost_of_living_index,
        cm.quality_of_life_index,
        RANK() OVER (PARTITION BY c.country 
                     ORDER BY cm.quality_of_life_index DESC) as country_rank,
        AVG(cm.cost_of_living_index) OVER (
            PARTITION BY c.country
        ) as avg_country_cost
    FROM cities c
    JOIN city_metrics cm ON c.id = cm.city_id
)
SELECT 
    city_name,
    country,
    cost_of_living_index,
    quality_of_life_index,
    country_rank,
    avg_country_cost,
    cost_of_living_index - avg_country_cost as cost_diff_from_avg
FROM RankedCities
ORDER BY quality_of_life_index DESC;
"""

# Execute the query
df_analysis = pd.read_sql(analysis_query, engine)
df_analysis

## 7. Visualization

Finally, let's create some visualizations of our analyzed data.

In [ ]:
# Set up the plotting style
plt.style.use('seaborn')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Quality of Life vs Cost of Living
sns.scatterplot(
    data=df_analysis,
    x='cost_of_living_index',
    y='quality_of_life_index',
    ax=ax1
)
ax1.set_title('Quality of Life vs Cost of Living')

# Plot 2: Cost difference from country average
sns.barplot(
    data=df_analysis,
    x='city_name',
    y='cost_diff_from_avg',
    ax=ax2
)
ax2.set_title('Cost of Living Difference from Country Average')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()